## Lab 4: Deploy to Production - Use AgentCore Runtime with Observability

### Overview

In Lab 3 we scaled our Customer Support Agent by centralizing tools through AgentCore Gateway with secure authentication. Now we'll complete the production journey by deploying our agent to AgentCore Runtime with comprehensive observability. This will transform our prototype into a production-ready system that can handle real-world traffic with full monitoring and automatic scaling.

[Amazon Bedrock AgentCore Runtime](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/agents-tools-runtime.html) is a secure, fully managed runtime that empowers organizations to deploy and scale AI agents in production, regardless of framework, protocol, or model choice. It provides enterprise-grade reliability, automatic scaling, and comprehensive monitoring capabilities.

**Workshop Journey:**

- **Lab 1 (Done):** Create Agent Prototype - Built a functional customer support agent
- **Lab 2 (Done):** Enhance with Memory - Added conversation context and personalization
- **Lab 3 (Done):** Scale with Gateway & Identity - Shared tools across agents securely
- **Lab 4 (Current):** Deploy to Production - Used AgentCore Runtime with observability
- **Lab 5:** Build User Interface - Create a customer-facing application

### Why AgentCore Runtime & Production Deployment Matter

Current State (Lab 1-3): Agent runs locally with centralized tools but faces production challenges:

- Agent runs locally in a single session
- No comprehensive monitoring or debugging capabilities
- Cannot handle multiple concurrent users reliably

After this lab, we will have a production-ready agent infrastructure with:

- Serverless auto-scaling to handle variable demand
- Comprehensive observability with traces, metrics, and logging
- Enterprise reliability with automatic error recovery
- Secure deployment with proper access controls
- Easy management through AWS console and APIs and support for real-world production workloads.


### Adding comprehensive observability with AgentCore Observability

Additionally, AgentCore Runtime integrates seamlessly with [AgentCore Observability](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/observability.html) to provide full visibility into your agent's behavior in production. AgentCore Observability automatically captures traces, metrics, and logs from your agent interactions, tool usage, and memory access patterns. In this lab we will see how AgentCore Runtime integrates with CloudWatch GenAI Observability to provide comprehensive monitoring and debugging capabilities.

For request tracing, AgentCore Observability captures the complete conversation flow including tool invocations, memory retrievals, and model interactions. For performance monitoring, it tracks response times, success rates, and resource utilization to help optimize your agent's performance.

During the observability flow, AgentCore Runtime automatically instruments your agent code and sends telemetry data to CloudWatch. You can then use CloudWatch dashboards and GenAI Observability features to analyze patterns, identify bottlenecks, and troubleshoot issues in real-time.

### Architecture for Lab 4
<div style="text-align:left"> 
    <img src="images/architecture_lab4_runtime.png" width="75%"/> 
</div>

*Agent now runs in AgentCore Runtime with full observability through CloudWatch, serving production traffic with auto-scaling and comprehensive monitoring. Memory and Gateway integrations from previous labs remain fully functional in the production environment.*

### Key Features

- **Serverless Agent Deployment:** Transform your local agent into a scalable production service using AgentCore Runtime with minimal code changes
- **Comprehensive Observability:** Full request tracing, performance metrics, and debugging capabilities through CloudWatch GenAI Observability

### Prerequisites

- Python 3.12+
- AWS account with appropriate permissions
- Docker, Finch or Podman installed and running
- Amazon Bedrock AgentCore SDK
- Strands Agents framework

**Note**: You MUST enable [CloudWatch Transaction Search](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/Enable-TransactionSearch.html) to be able to see AgentCore Observability traces in CloudWatch.


### Step 1: Import Required Libraries

In [1]:
# Import required libraries
import os
import json
import boto3
from strands import Agent
from strands.models import BedrockModel
from lab_helpers.lab2_memory import create_or_get_memory_resource

create_or_get_memory_resource()  # Just in case the memory lab wasn't executed

'TravelAgentMemory-jEgT5dAIC4'

### Step 2: Preparing Your Agent for AgentCore Runtime

#### Creating the Runtime-Ready Agent

Let's first define the necessary AgentCore Runtime components via Python SDK within our previous local agent implementation.

Observe the `#### AGENTCORE RUNTIME - LINE i ####` comments below to see where is the relevant deployment code added. You'll find 4 such lines that prepare the runtime-ready agent:

1. Import the Runtime App with `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
2. Initialize the App with `app = BedrockAgentCoreApp()`
3. Decorate our invocation function with `@app.entrypoint`
4. Let AgentCore Runtime control the execution with `app.run()`


In [2]:
%%writefile ./lab_helpers/lab4_runtime.py
from bedrock_agentcore.runtime import (
    BedrockAgentCoreApp,
)  #### AGENTCORE RUNTIME - LINE 1 ####
from strands import Agent
from strands.models import BedrockModel
from scripts.utils import get_ssm_parameter


from lab_helpers.lab1_strands_agent import (
    get_destination_info,
    get_experience_info,
    get_signals,
    SYSTEM_PROMPT,
    MODEL_ID,
)

from lab_helpers.lab2_memory import (
    CustomerSupportMemoryHooks,
    memory_client,
    ACTOR_ID,
    SESSION_ID,
)

# Lab1 import: Create the Bedrock model
print('model:', MODEL_ID)
model = BedrockModel(model_id=MODEL_ID)

# Lab2 import : Initialize memory via hooks
memory_id = get_ssm_parameter("/app/travelagent/agentcore/memory_id")
memory_hooks = TravelAgentMemoryHooks(
    memory_id, memory_client, ACTOR_ID, SESSION_ID
)

# Create the agent with all customer support tools
agent = Agent(
    model=model,
    tools=[get_destination_info, get_experience_info, get_signals],
    system_prompt=SYSTEM_PROMPT,
    hooks=[memory_hooks],
)

# Initialize the AgentCore Runtime App
app = BedrockAgentCoreApp()  #### AGENTCORE RUNTIME - LINE 2 ####


@app.entrypoint  #### AGENTCORE RUNTIME - LINE 3 ####
def invoke(payload):
    """AgentCore Runtime entrypoint function"""
    user_input = payload.get("prompt", "")

    # Invoke the agent
    response = agent(user_input)
    return response.message["content"][0]["text"]


if __name__ == "__main__":
    app.run()  #### AGENTCORE RUNTIME - LINE 4 ####

Overwriting ./lab_helpers/lab4_runtime.py


#### What happens behind the scenes?

When you use `BedrockAgentCoreApp`, it automatically:

- Creates an HTTP server that listens on port 8080
- Implements the required `/invocations` endpoint for processing requests
- Implements the `/ping` endpoint for health checks
- Handles proper content types and response formats
- Manages error handling according to AWS standards


### Step 3: Deploying to AgentCore Runtime

Now let's deploy our agent to AgentCore Runtime using the [AgentCore Starter Toolkit](https://github.com/aws/bedrock-agentcore-starter-toolkit).

#### Configure the Secure Runtime Deployment (AgentCore Runtime + AgentCore Identity)

First we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we will create and a requirements file. We will also configure the identity authorization using an Amazon Cognito user pool and we will configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

<div style="text-align:left"> 
    <img src="images/configure.png" width="75%"/> 
</div>

**Note**: The Cognito access_token is valid for 2 hours only. If the access_token expires you can vend another access_token by using the `reauthenticate_user` method.


In [4]:
from lab_helpers.utils import setup_cognito_user_pool, reauthenticate_user

print("Setting up Amazon Cognito user pool...")
cognito_config = (
    setup_cognito_user_pool()
)  # You'll get your bearer token from this output cell.
print("Cognito setup completed ✓")

Setting up Amazon Cognito user pool...
{'UserPoolId': 'ap-southeast-2_V3bb62LQH', 'ClientName': 'MCPServerPoolClient', 'ClientId': '7slmcn3dssnnkag650fklgi954', 'ClientSecret': '1r7eif2atokd8lb4vt74cqr22k6dmft6eok34h3nrfkqboa9phjb', 'LastModifiedDate': datetime.datetime(2025, 9, 23, 14, 23, 45, 81000, tzinfo=tzlocal()), 'CreationDate': datetime.datetime(2025, 9, 23, 14, 23, 45, 81000, tzinfo=tzlocal()), 'RefreshTokenValidity': 30, 'TokenValidityUnits': {}, 'ExplicitAuthFlows': ['ALLOW_USER_PASSWORD_AUTH', 'ALLOW_USER_SRP_AUTH', 'ALLOW_REFRESH_TOKEN_AUTH'], 'AllowedOAuthFlowsUserPoolClient': False, 'EnableTokenRevocation': True, 'EnablePropagateAdditionalUserContextData': False, 'AuthSessionValidity': 3}
Pool id: ap-southeast-2_V3bb62LQH
Discovery URL: https://cognito-idp.ap-southeast-2.amazonaws.com/ap-southeast-2_V3bb62LQH/.well-known/openid-configuration
Client ID: 7slmcn3dssnnkag650fklgi954
Bearer Token: eyJraWQiOiIyRktpVk9zbjh0cjk3R2pHYVhOWDlTZksxdDNNVWI0S2RQK1p1c0NKXC9EYz0iLCJhbGc

In [5]:
from bedrock_agentcore_starter_toolkit import Runtime
from lab_helpers.utils import create_agentcore_runtime_execution_role

# Initialize the runtime toolkit
boto_session = boto3.session.Session()
region = boto_session.region_name

execution_role_arn = create_agentcore_runtime_execution_role()

agentcore_runtime = Runtime()

# Configure the deployment
response = agentcore_runtime.configure(
    entrypoint="lab_helpers/lab4_runtime.py",
    execution_role=execution_role_arn,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="customer_support_agent",
    authorizer_configuration={
        "customJWTAuthorizer": {
            "allowedClients": [cognito_config.get("client_id")],
            "discoveryUrl": cognito_config.get("discovery_url"),
        }
    },
)

print("Configuration completed:", response)

Entrypoint parsed: file=/Users/mike/Documents/poplin/amazon-bedrock-agentcore-samples/01-tutorials/07-AgentCore-E2E/lab_helpers/lab4_runtime.py, bedrock_agentcore_name=lab4_runtime
Configuring BedrockAgentCore agent: customer_support_agent


ℹ️ Role CustomerSupportAssistantBedrockAgentCoreRole-ap-southeast-2 already exists
Role ARN: arn:aws:iam::485941242585:role/CustomerSupportAssistantBedrockAgentCoreRole-ap-southeast-2


Generated Dockerfile: /Users/mike/Documents/poplin/amazon-bedrock-agentcore-samples/01-tutorials/07-AgentCore-E2E/Dockerfile
Generated .dockerignore: /Users/mike/Documents/poplin/amazon-bedrock-agentcore-samples/01-tutorials/07-AgentCore-E2E/.dockerignore
Keeping 'customer_support_agent' as default agent
Bedrock AgentCore configured: /Users/mike/Documents/poplin/amazon-bedrock-agentcore-samples/01-tutorials/07-AgentCore-E2E/.bedrock_agentcore.yaml


Configuration completed: config_path=PosixPath('/Users/mike/Documents/poplin/amazon-bedrock-agentcore-samples/01-tutorials/07-AgentCore-E2E/.bedrock_agentcore.yaml') dockerfile_path=PosixPath('/Users/mike/Documents/poplin/amazon-bedrock-agentcore-samples/01-tutorials/07-AgentCore-E2E/Dockerfile') dockerignore_path=PosixPath('/Users/mike/Documents/poplin/amazon-bedrock-agentcore-samples/01-tutorials/07-AgentCore-E2E/.dockerignore') runtime='Docker' region='ap-southeast-2' account_id='485941242585' execution_role='arn:aws:iam::485941242585:role/CustomerSupportAssistantBedrockAgentCoreRole-ap-southeast-2' ecr_repository=None auto_create_ecr=True


#### Launch the Agent

Now let's launch our agent to AgentCore Runtime. This will create an AWS CodeBuild pipeline, the Amazon ECR repository and the AgentCore Runtime components.

<div style="text-align:left"> 
    <img src="images/launch.png" width="100%"/> 
</div>

In [6]:
# Launch the agent (this will build and deploy the container)
from lab_helpers.utils import put_ssm_parameter

launch_result = agentcore_runtime.launch()
print("Launch completed:", launch_result.agent_arn)

agent_arn = put_ssm_parameter(
    "/app/customersupport/agentcore/runtime_arn", launch_result.agent_arn
)

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'customer_support_agent' to account 485941242585 (ap-southeast-2)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: customer_support_agent
✅ ECR repository available: 485941242585.dkr.ecr.ap-southeast-2.amazonaws.com/bedrock-agentcore-customer_support_agent
Using execution role from config: arn:aws:iam::485941242585:role/CustomerSupportAssistantBedrockAgentCoreRole-ap-southeast-2
Preparing CodeBuild project and uploading source...


✅ Reusing existing ECR repository: 485941242585.dkr.ecr.ap-southeast-2.amazonaws.com/bedrock-agentcore-customer_support_agent


Getting or creating CodeBuild execution role for agent: customer_support_agent
Role name: AmazonBedrockAgentCoreSDKCodeBuild-ap-southeast-2-aa96c3a063
Reusing existing CodeBuild execution role: arn:aws:iam::485941242585:role/AmazonBedrockAgentCoreSDKCodeBuild-ap-southeast-2-aa96c3a063
Using .dockerignore with 44 patterns
Uploaded source to S3: customer_support_agent/source.zip
Updated CodeBuild project: bedrock-agentcore-customer_support_agent-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 1.1s
🔄 PROVISIONING started (total: 1s)
✅ PROVISIONING completed in 7.5s
🔄 DOWNLOAD_SOURCE started (total: 9s)
✅ DOWNLOAD_SOURCE completed in 1.1s
🔄 BUILD started (total: 10s)
✅ BUILD completed in 17.0s
🔄 POST_BUILD started (total: 27s)
✅ POST_BUILD completed in 10.6s
🔄 FINALIZING started (total: 37s)
✅ FINALIZING completed in 1.1s
🔄 COMPLETED started (total: 38s)
✅ COMPLETED completed in 0.0s
🎉 C

Launch completed: arn:aws:bedrock-agentcore:ap-southeast-2:485941242585:runtime/customer_support_agent-QmbieiB510


#### Check Deployment Status

Let's wait for the deployment to complete:


In [7]:
import time

# Wait for the agent to be ready
status_response = agentcore_runtime.status()
status = status_response.endpoint["status"]

end_status = ["READY", "CREATE_FAILED", "DELETE_FAILED", "UPDATE_FAILED"]
while status not in end_status:
    print(f"Waiting for deployment... Current status: {status}")
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint["status"]

print(f"Final status: {status}")

Retrieved Bedrock AgentCore status for: customer_support_agent


Final status: READY


### Step 4: Invoking Your Deployed Agent

Now that our agent is deployed and ready, let's test it with some queries. We invoke the agent with the right authorization token type. In out case it'll be Cognito access token. Copy the access token from the cell above

<div style="text-align:left"> 
    <img src="images/invoke.png" width="100%"/> 
</div>

#### Using the AgentCore Starter Toolkit

We can validate that the agent works using the AgentCore Starter Toolkit for invocation. The starter toolkit can automatically create a session id for us to query our agent. Alternatively, you can also pass the session id as a parameter during invocation. For demonstration purpose, we will create our own session id.

In [9]:
import uuid

def old():
    # Create a session ID for demonstrating session continuity
    session_id = uuid.uuid4()
    
    # Test different customer support scenarios
    user_query = "My Iphone is not connecting with the Bluetooth. What should I do?"
    
    bearer_token = reauthenticate_user(
        cognito_config.get("client_id"), 
        cognito_config.get("client_secret")
    )
    
    print('tok:', bearer_token)
    
    response = agentcore_runtime.invoke(
        {"prompt": user_query}, 
        bearer_token=bearer_token,
        session_id=str(session_id)
    )
    print(response)
    response

# mike custom

old()

tok: eyJraWQiOiIyRktpVk9zbjh0cjk3R2pHYVhOWDlTZksxdDNNVWI0S2RQK1p1c0NKXC9EYz0iLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiIxOTZlMzRiOC0xMGExLTcwZWUtNjk3ZC0yYWNkNzY5MmFhMmMiLCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAuYXAtc291dGhlYXN0LTIuYW1hem9uYXdzLmNvbVwvYXAtc291dGhlYXN0LTJfVjNiYjYyTFFIIiwiY2xpZW50X2lkIjoiN3NsbWNuM2Rzc25ua2FnNjUwZmtsZ2k5NTQiLCJvcmlnaW5fanRpIjoiYzgwNGM4MjYtNzExNy00MGJmLWI1M2YtZjVmZTQwOTE4NjhjIiwiZXZlbnRfaWQiOiI2NDgzMWRiNi1jOTYwLTRmN2QtYmM2OC1hYTM2NjgwYzMwOTQiLCJ0b2tlbl91c2UiOiJhY2Nlc3MiLCJzY29wZSI6ImF3cy5jb2duaXRvLnNpZ25pbi51c2VyLmFkbWluIiwiYXV0aF90aW1lIjoxNzU4NjAxNjAxLCJleHAiOjE3NTg2MDUyMDEsImlhdCI6MTc1ODYwMTYwMSwianRpIjoiOTNjNjJlMGYtZTVmOC00YTk0LWI5MjMtNTU2NWQ4NzlkYmYxIiwidXNlcm5hbWUiOiJ0ZXN0dXNlciJ9.jymGk0Re0B6HPAhS9FNRw9W3LFoIuCW-WO1p67Eix0MDS79ucv6CDuhEDL4jrYTvN0zdeJ9AWTSoZmmyJmyoeAFl4x8NQJuWPenXtPef3hWLAl9aGJpnf6viIR7P40Uu9KQmy46fNJ3vVHdm46ShQ4zWtDo_iQRzh6ciEre4JDu3aLibkFupdtt018ebBmhrX6Jb9SLQgsRcaXUdpuA8rhAsb_nPV2Dw_NP6NK51UNYTyIU2CrpoS10ikdXUDYwxa-5ksUiww7-eRXRnbBruaVf9GJr79kzfJ

#### Invoking the agent with session continuity

Since we are using AgentCore Runtime, we can easily continue our conversation with the same session id.

In [9]:
user_query = "I've turned my Bluetooth on and off but it still does not work"
response = agentcore_runtime.invoke(
    {"prompt": user_query}, 
    bearer_token=bearer_token,
    session_id=str(session_id)
)
response

Failed to invoke agent endpoint: 424 Client Error: Failed Dependency for url: https://bedrock-agentcore.ap-southeast-2.amazonaws.com/runtimes/arn%3Aaws%3Abedrock-agentcore%3Aap-southeast-2%3A485941242585%3Aruntime%2Fcustomer_support_agent-QmbieiB510/invocations?qualifier=DEFAULT


HTTPError: 424 Client Error: Failed Dependency for url: https://bedrock-agentcore.ap-southeast-2.amazonaws.com/runtimes/arn%3Aaws%3Abedrock-agentcore%3Aap-southeast-2%3A485941242585%3Aruntime%2Fcustomer_support_agent-QmbieiB510/invocations?qualifier=DEFAULT

#### Invoking the agent with a new user
In the example below we have not mentioned the Iphone device in the second query, but our agent still has the context of it. This is due to the AgentCore Runtime session continuity. The agent won't know the context for a new user.

In [ ]:
# Creating a new session ID for demonstrating new customer
session_id2 = uuid.uuid4()

user_query = "Still not working. What is going on?"
response = agentcore_runtime.invoke(
    {"prompt": user_query}, 
    bearer_token=bearer_token,
    session_id=str(session_id2)
)
response

In this case our agent does not have the context anymore and needs more information. 

And it is all it takes to have a secure and scalable endpoint for our Agent with no need to manage all the underlying infrastructure!

### Step 5: AgentCore Observability

[AgentCore Observability](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/observability.html) provides monitoring and tracing capabilities for AI agents using Amazon OpenTelemetry Python Instrumentation and Amazon CloudWatch GenAI Observability.

#### Agents

Default AgentCore Runtime configuration allows for logging our agent's traces on CloudWatch by means of **AgentCore Observability**. These traces can be seen on the AWS CloudWatch GenAI Observability dashboard. Navigate to Cloudwatch &rarr; GenAI Observability &rarr; Bedrock AgentCore.

![Agents Overview on CloudWatch](images/observability_agents.png)

#### Sessions

The Sessions view shows the list of all the sessions associated with all agents in your account.

![sessions](images/sessions_lab5_observability.png)

#### Traces

Trace view lists all traces from your agents in this account. To work with traces:

- Choose Filter traces to search for specific traces.
- Sort by column name to organize results.
- Under Actions, select Logs Insights to refine your search by querying across your log and span data or select Export selected traces to export.

![traces](images/traces_lab4_observability.png)


### Congratulations! 🎉

You have successfully completed **Lab 4: Deploy to Production - Use AgentCore Runtime with Observability!**

Here is what you accomplished:

##### Production-Ready Deployment:

- Prepared your agent for production with minimal code changes (only 4 lines added)
- Validated proper session isolation between different customers
- Confirmed session continuity + memory persistence and context awareness per session

##### Enterprise-Grade Security & Identity:

- Implemented secure authentication using Cognito integration with JWT tokens
- Configured proper IAM roles and execution permissions for production workloads
- Established identity-based access control for secure agent invocation

##### Comprehensive Observability:

- Enabled AgentCore Observability for full request tracing across all customer sessions
- Configured CloudWatch GenAI Observability dashboard monitoring

##### Current Limitations (We'll fix these next!):

- **Developer Focused Interaction** - Agent accessible via SDK/API calls but no user-friendly web interface
- **Manual Session Management** - Requires programmatic session creation rather than intuitive user experience

##### Next Up [Lab 5: Build User Interface →](lab-05-frontend.ipynb)
In Lab 5, you'll complete the customer experience by building a user-friendly interface !! Lets go !!


In [13]:
import requests
import urllib.parse
from uuid import uuid4

REGION_NAME = 'ap-southeast-2'
invoke_agent_arn = 'arn:aws:bedrock-agentcore:ap-southeast-2:485941242585:runtime/customer_support_agent-QmbieiB510'
escaped_agent_arn = urllib.parse.quote(invoke_agent_arn, safe='')

# Construct the URL
url = f"https://bedrock-agentcore.{REGION_NAME}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations?qualifier=DEFAULT"
print(url)
# url = 'https://bedrock-agentcore.ap-southeast-2.amazonaws.com/runtimes/arn%3Aaws%3Abedrock-agentcore%3Aap-southeast-2%3A485941242585%3Aruntime%2Fcustomer_support_agent-QmbieiB510/invocations?qualifier=DEFAULT'

sid = str(uuid4())
print(sid)

bearer_token = reauthenticate_user(
    cognito_config.get("client_id"), 
    cognito_config.get("client_secret")
)

headers = {
    'Authorization': f'Bearer {bearer_token}',
    'Content-Type': 'application/json',
    "X-Amzn-Bedrock-AgentCore-Runtime-Session-Id": sid
}

invoke_response = requests.post(
    url,
    headers=headers,
    data=json.dumps({"prompt": "what LLM model are you using?"})
)
print(invoke_response.json())
    

https://bedrock-agentcore.ap-southeast-2.amazonaws.com/runtimes/arn%3Aaws%3Abedrock-agentcore%3Aap-southeast-2%3A485941242585%3Aruntime%2Fcustomer_support_agent-QmbieiB510/invocations?qualifier=DEFAULT
312de76c-697d-4a1c-ba28-0067e6dd3fbe
I understand you're asking about the LLM (Large Language Model) that I'm using. I'm Claude, an AI assistant created by Anthropic. I'm designed to be helpful, harmless, and honest in my interactions.

If you have any specific questions about electronics products, return policies, or technical support, I'd be happy to assist you using the tools available to me. Based on your customer context, I see you have interests in ThinkPad laptops with Linux compatibility and gaming headphones with low latency for competitive FPS games. I can provide information about these types of products if you'd like.

Is there something specific about electronics that I can help you with today?


In [18]:
# Print response in a safe manner
print(f"Status Code: {invoke_response.status_code}")
print(f"Response Headers: {dict(invoke_response.headers)}")

Status Code: 400
Response Headers: {'Date': 'Tue, 23 Sep 2025 04:18:52 GMT', 'Content-Type': 'application/json', 'Content-Length': '183', 'Connection': 'keep-alive', 'x-amzn-RequestId': '1529d946-c11b-4b7e-be82-09055169e933', 'x-amzn-ErrorType': 'ValidationException:http://internal.amazon.com/coral/com.amazon.bedrock.agentcore.common/'}


In [15]:
bearer_token = reauthenticate_user(
    cognito_config.get("client_id"), 
    cognito_config.get("client_secret")
)
print(cognito_config)

{'pool_id': 'ap-southeast-2_V3bb62LQH', 'client_id': '7slmcn3dssnnkag650fklgi954', 'client_secret': '1r7eif2atokd8lb4vt74cqr22k6dmft6eok34h3nrfkqboa9phjb', 'secret_hash': 'dRYmS9GQ9HO7eMzUJWJoFraemUiJy20x19JMRaCDopo=', 'bearer_token': 'eyJraWQiOiIyRktpVk9zbjh0cjk3R2pHYVhOWDlTZksxdDNNVWI0S2RQK1p1c0NKXC9EYz0iLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiIxOTZlMzRiOC0xMGExLTcwZWUtNjk3ZC0yYWNkNzY5MmFhMmMiLCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAuYXAtc291dGhlYXN0LTIuYW1hem9uYXdzLmNvbVwvYXAtc291dGhlYXN0LTJfVjNiYjYyTFFIIiwiY2xpZW50X2lkIjoiN3NsbWNuM2Rzc25ua2FnNjUwZmtsZ2k5NTQiLCJvcmlnaW5fanRpIjoiODExMTZjMzUtYWUwOC00YmJhLTk4NWQtZDJiODgzMTY0ZjU1IiwiZXZlbnRfaWQiOiJmODU3MWVhZS1hM2E3LTQwNzktODZiOC1iZDQyYzQ0OTU3NWMiLCJ0b2tlbl91c2UiOiJhY2Nlc3MiLCJzY29wZSI6ImF3cy5jb2duaXRvLnNpZ25pbi51c2VyLmFkbWluIiwiYXV0aF90aW1lIjoxNzU4NjAxNDI2LCJleHAiOjE3NTg2MDUwMjYsImlhdCI6MTc1ODYwMTQyNiwianRpIjoiNmZhZTNmNTEtYjU3MC00NmM5LTg5NzctYjFhMWM2NDZkYmU2IiwidXNlcm5hbWUiOiJ0ZXN0dXNlciJ9.tHhayzEa_AcstQyNA8bjFcKH_UulO0-hGms01l_v9qxOhH7BcjNenS2GS3